```--- Day 18: Duet ---```

In [1]:
import unittest
import re
from time import sleep

In [2]:
test_prog = '''set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2'''.split('\n')

prog = open('input.txt').readlines()


    snd X plays a sound with a frequency equal to the value of X.
    set X Y sets register X to the value of Y.
    add X Y increases register X by the value of Y.
    mul X Y sets register X to the result of multiplying the value contained in register X by the value of Y.
    mod X Y sets register X to the remainder of dividing the value contained in register X by the value of Y (that is, it sets X to the result of X modulo Y).
    rcv X recovers the frequency of the last sound played, but only when the value of X is not zero. (If it is zero, the command does nothing.)
    jgz X Y jumps with an offset of the value of Y, but only if the value of X is greater than zero. (An offset of 2 skips the next instruction, an offset of -1 jumps to the previous instruction, and so on.)


In [3]:
class VM(object):
    def __init__(self, progl, prognum=0, part=1, debug=False):
        self.prog = progl[:]
        self.prognum = prognum
        self.part = part
        self.debug = debug
        self.registers = {'p': prognum}
        self.pc = 0
        self.running = False
        self.blocked = [False]
        self.otherblocked = self.blocked
        self.snds = [] # all sounds played, earliest first
        self.sndcount = 0
        self.rcvs = self.snds

    def connect(self, other):
        self.rcvs = other.snds
        self.otherblocked = other.blocked
        
    def run(self):
        self.running = True
        while self.running:
            self.step()
        print(f' <P{self.prognum}> stopped')
        return self
    
    linematcher = re.compile(r'^([^ ]+) ([^ ]+)( ([^ ]+))?')

    def step(self):
        try:
            l = self.prog[self.pc].strip()
        except IndexError:
            self.running = False
            return self
        instr, op1, op2, _unused = VM.linematcher.match(l).groups()
        f, args = VM.dispatcher[instr]
        if args == 1:
            f(self, op1)
        else:
            f(self, op1, op2)
        if self.debug:
            print(f' <P{self.prognum}> {self.pc:4}: {l} | {self.registers}', flush=True)
        if self.blocked[0] & self.otherblocked[0]: # RACE CONDITION?
            self.running = False
            print(f'!! P{self.prognum} detected DEADLOCK !!')
        return self
    
    def _snd(self, op):
        self.sndcount += 1
        self.snds.append(self.value(op))
        self.otherblocked[0] = False
        self.pc += 1

    def _rcv(self, op):
        if self.part == 1:
            # last sound played / message sent
            print(f'part 1 answer: {self.snds[-1]}')
            self.running = False
        while True:
            try:
                val = self.rcvs.pop(0) # pull the oldest sound/message
                self.blocked[0] = False
                break
            except IndexError:
                # run hot (could also sleep() a bit here)
                self.blocked[0] = True
                return # doesn't change self.pc, so instruction will be retried
        self.set_reg(op, val)
        self.pc += 1
        
    def _set(self, op1, op2):
        self.set_reg(op1, self.value(op2))
        self.pc += 1

    def _add(self, op1, op2):
        self.set_reg(op1, self.value(op1) + self.value(op2))
        self.pc += 1
        
    def _mul(self, op1, op2):
        self.set_reg(op1, self.value(op1) * self.value(op2))
        self.pc += 1
        
    def _mod(self, op1, op2):
        self.set_reg(op1, self.value(op1) % self.value(op2))
        self.pc += 1
    
    def _jgz(self, op1, op2):
        if self.value(op1) > 0:
            self.pc += self.value(op2)
        else:
            self.pc += 1

    dispatcher = {'snd': (_snd, 1), # 'instr': (_instr_func, args)
                  'rcv': (_rcv, 1),
                  'set': (_set, 2),
                  'add': (_add, 2),
                  'mul': (_mul, 2),
                  'mod': (_mod, 2),
                  'jgz': (_jgz, 2)}
            
    def value(self, v):
        v = v.strip()
        try:
            return int(v)
        except ValueError:
            return self.get_reg(v)
            
    def get_reg(self, r):
        r = r.strip()
        if r in self.registers:
            return self.registers[r]
        else:
            self.registers[r] = 0
            return 0
        
    def set_reg(self, r, val):
        self.registers[r] = val

# Part 1 Test

In [4]:
v = VM(test_prog)

In [5]:
v.run()

part 1 answer: 4
 <P0> stopped


# Part 1 Actual

In [6]:
v = VM(prog)

In [7]:
v.run()

part 1 answer: 7071
 <P0> stopped


# Part 2

In [8]:
USE_THREADS = True

In [9]:
test_prog2 = '''snd 1
snd 2
snd p
rcv a
rcv b
rcv c
rcv d'''.split('\n')

# With Threads:-

In [10]:
if USE_THREADS:
    # test
    # prog0 = VM(test_prog2, prognum=0, part=2, debug=True)
    # prog1 = VM(test_prog2, prognum=1, part=2, debug=True)

    # actual
    prog0 = VM(prog, prognum=0, part=2, debug=False)
    prog1 = VM(prog, prognum=1, part=2, debug=False)

    prog0.connect(prog1)
    prog1.connect(prog0)

    import threading

    threads = [threading.Thread(target=prog0.run), threading.Thread(target=prog1.run)]

    [t.start() for t in threads]
    [t.join() for t in threads]

    print(f'\n\npart 2 answer: {prog1.sndcount}')
else:
    print('Not using threads this time')

!! P1 detected DEADLOCK !!!! P0 detected DEADLOCK !!


part 2 answer: 8001
 <P1> stopped
 <P0> stopped



# Without Threads:-

In [11]:
# test
# prog0 = VM(test_prog2, prognum=0, part=2, debug=True)
# prog1 = VM(test_prog2, prognum=1, part=2, debug=True)

# actual
prog0 = VM(prog, prognum=0, part=2, debug=False)
prog1 = VM(prog, prognum=1, part=2, debug=False)

In [12]:
prog0.connect(prog1)
prog1.connect(prog0)

In [13]:
iprog = 0
progs = [prog0, prog1]
while True:
    p = progs[iprog]
    #print(f'P{p.prognum} ', flush=True, end='')
    while not p.blocked[0]:
        #print('.', flush=True, end='')
        p.step()
    iprog = 1 - iprog
    #print('\n')
    if all([sp.blocked[0] for sp in progs]):
        print('!! Scheduler detected DEADLOCK !!')
        break

    

!! P1 detected DEADLOCK !!
!! Scheduler detected DEADLOCK !!


In [14]:
print(f'\n\npart 2 answer: {prog1.sndcount}')



part 2 answer: 8001
